# How to deal with timestamps

## Install

In [1]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# !pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [2]:
import syft as sy
sy.requires(SYFT_VERSION)

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/PySyft


✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8.1b0 and the requirement <0.9


In [3]:
node = sy.orchestra.launch(name="pandas-test-domain-1", reset=True)

SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/281e55cb3425360e464682cfb4672fcb.sqlite



# Data owner: upload dataset

In [4]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

## Load data

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at /var/log/popularity-contest.

In [5]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from syft.service.project.project import Project
from syft.util.util import autocache, PANDAS_DATA

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 3)
plt.rcParams['font.family'] = 'sans-serif'

In [6]:
popcon = pd.read_csv(autocache(f"{PANDAS_DATA}/popularity-contest"), sep=' ', )[:-1]
popcon.columns = ['atime', 'ctime', 'package-name', 'mru-program', 'tag']

In [7]:
set(popcon["tag"])

{'<OLD>', '<RECENT-CTIME>', nan}

## Create mock data

Lets create a mock dataset

In [8]:
# weather_types = set(weather_2012_final.Weather.to_list())

In [9]:
from random import randint
import random

In [10]:
def get_random_timestamp():
    return "135" + "".join([str(randint(0,9)) for i in range(6)])

In [11]:
def get_mock_date(i):
    return str(parse('Jun 1 2010') + timedelta(days=i))

def get_mock_row(i):
    res = dict()
    for k, function in mock_functions.items():
        res[k] = function()
    return res
    

In [12]:
mock_functions = {'atime': lambda: get_random_timestamp(),
                  'ctime': lambda: get_random_timestamp(),
                  'package-name': lambda: random.choice([
                      'libghc-stm-dev',
                      'libqtdee-dev',
                      'geoclue-ubuntu-geoip',
                      'libdesktop-agnostic0',
                      'ubuntu-extras-keyring',
                      'libbsd0',
                      'libxres-dev']),
                  'mru-program': lambda: random.choice([
                      '/usr/bin/opam',
                      '/usr/bin/onboard',
                      '/lib/init/upstart-job',
                      '/usr/bin/page',
                  ]),
                  'tag': lambda: random.choice(['<OLD>', '<RECENT-CTIME>', 'nan'])}


In [13]:
mock = pd.DataFrame(data=[get_mock_row(i) for i in range(len(popcon))],
                    columns=popcon.columns)
    


Upload the data

In [14]:
dataset = sy.Dataset(name="test", asset_list=[sy.Asset(name=f"weather", data=popcon,
                                                       mock=mock, mock_is_real=False)])
root_domain_client.upload_dataset(dataset)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.02it/s]


Uploading: weather


<class 'syft.service.response.SyftSuccess'>: Dataset Added

In [15]:
popcon.head()

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,NaN
1,1387295796,1354370480,login,/bin/su,NaN
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,NaN
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,NaN


## Create user account

In [16]:
user = root_domain_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")
# todo: give user data scientist role
guest_domain_client = node.client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

# Data scientist: create result pointer

In [17]:
import numpy as np
import pandas as pd

## Summary

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at `/var/log/popularity-contest`.

## Get mocks

In [18]:
ds = guest_domain_client.datasets[0]

In [19]:
asset = ds.assets[0]

In [21]:
popcon = asset.mock

## Parsing Unix timestamps

The colums are the access time, created time, package name, recently used program, and a tag



In [22]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,135653948,135100299,ubuntu-extras-keyring,/usr/bin/opam,<RECENT-CTIME>
1,135102022,135377811,libqtdee-dev,/usr/bin/onboard,<RECENT-CTIME>
2,135050066,135957018,libghc-stm-dev,/usr/bin/onboard,<RECENT-CTIME>
3,135240285,135990066,ubuntu-extras-keyring,/usr/bin/page,<RECENT-CTIME>
4,135892534,135581665,libdesktop-agnostic0,/usr/bin/onboard,<RECENT-CTIME>


The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [23]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [25]:
popcon['atime'] = guest_client.api.lib.pandas.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = guest_client.api.lib.pandas.to_datetime(popcon['ctime'], unit='s')

If we look at the dtype now, it's `<M8[ns]`. As far as I can tell `M8` is secret code for `datetime64`.

In [26]:
popcon['atime'].dtype

```python
Pointer
```
dtype('<M8[ns]')

So now we can look at our `atime` and `ctime` as dates!

In [27]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,1974-04-20 01:39:08,1974-04-13 15:51:39,ubuntu-extras-keyring,/usr/bin/opam,<RECENT-CTIME>
1,1974-04-13 16:20:22,1974-04-16 20:56:51,libqtdee-dev,/usr/bin/onboard,<RECENT-CTIME>
2,1974-04-13 01:54:26,1974-04-23 13:50:18,libghc-stm-dev,/usr/bin/onboard,<RECENT-CTIME>
3,1974-04-15 06:44:45,1974-04-23 23:01:06,ubuntu-extras-keyring,/usr/bin/page,<RECENT-CTIME>
4,1974-04-22 19:55:34,1974-04-19 05:34:25,libdesktop-agnostic0,/usr/bin/onboard,<RECENT-CTIME>


Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.


In [28]:
popcon = popcon[popcon['atime'] > '1970-01-01']

Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [29]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

/Users/koen/workspace/PySyft/packages/syft/src/syft/service/action/action_service.py:626: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = target_method(*private_args, **private_kwargs)
/Users/koen/workspace/PySyft/packages/syft/src/syft/service/action/action_service.py:633: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = target_method(*mock_args, **mock_kwargs)


In [30]:
nonlibraries.sort_values('ctime', ascending=False)[:10]

,atime,ctime,package-name,mru-program,tag
2429,1974-04-21 07:38:42,1974-04-24 00:51:35,geoclue-ubuntu-geoip,/lib/init/upstart-job,<OLD>
15,1974-04-14 20:18:08,1974-04-24 00:44:31,ubuntu-extras-keyring,/lib/init/upstart-job,<RECENT-CTIME>
65,1974-04-15 03:48:40,1974-04-24 00:34:00,ubuntu-extras-keyring,/usr/bin/opam,<RECENT-CTIME>
129,1974-04-17 12:10:50,1974-04-23 23:25:40,geoclue-ubuntu-geoip,/usr/bin/onboard,<RECENT-CTIME>
1445,1974-04-22 22:14:00,1974-04-23 23:24:52,geoclue-ubuntu-geoip,/usr/bin/opam,nan
548,1974-04-17 20:32:32,1974-04-23 23:12:32,geoclue-ubuntu-geoip,/usr/bin/onboard,<RECENT-CTIME>
3,1974-04-15 06:44:45,1974-04-23 23:01:06,ubuntu-extras-keyring,/usr/bin/page,<RECENT-CTIME>
93,1974-04-15 04:27:20,1974-04-23 22:58:27,geoclue-ubuntu-geoip,/usr/bin/page,nan
639,1974-04-23 17:29:56,1974-04-23 22:51:39,geoclue-ubuntu-geoip,/usr/bin/opam,<OLD>
174,1974-04-19 15:24:15,1974-04-23 22:00:50,geoclue-ubuntu-geoip,/usr/bin/onboard,<OLD>


Okay, cool, it says that I I installed ddd recently. And postgresql! I remember installing those things. Neat.

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

## Requesting result

In [31]:
nonlibraries.request(guest_client)

```python
class Request:
  id: str = 739c91bf8fea4030b62ba10d880500df
  requesting_user_verify_key: str = 1f670aef2b4ee98d980264dabe7446269aa873a490cd924c844579a6527824ac
  approving_user_verify_key: str = None
  request_time: str = 2023-05-22 07:12:54
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 281e55cb3425360e464682cfb4672fcb
  request_hash: str = "eccb151507802b8363a316b51dc0211bbe7c93c375fa8af87285ca253250caa9"
  changes: str = [syft.service.request.request.ActionStoreChange]

```

# Data owner: accept request

In [32]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

In [34]:
root_domain_client.api.services.request[0].approve()

<class 'syft.service.response.SyftSuccess'>: Request 739c91bf8fea4030b62ba10d880500df changes applied

# Data Owner: fetch result

In [35]:
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [36]:
real_result = nonlibraries.get_from(guest_client)

In [37]:
real_result

,atime,ctime,package-name,mru-program,tag


In [38]:
node.land()